In [ ]:
!pip install tensorflow


In [ ]:
import joblib
import numpy as np
from scipy.sparse import hstack, csr_matrix
import ast
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
model = joblib.load("toxicity_model.pkl")
tfidf = joblib.load("Preprocessing.pkl")

df = pd.read_csv("cleaned_and_embed_comments.csv")



/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.4.2 when using version 1.6.1. This might lead to

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.columns

Index(['user', 'post_id', 'comment', 'upvotes', 'responding_to',
       'responding_to_id', 'comment_id', 'links', 'has_sarcasm', 'has_joke',
       'bert_embed', 'preprocessed_text', 'tfidf_embed', 'Title_x',
       'text_sentiment', 'sentiment_difference', 'entity_sentiments', 'fear',
       'anger', 'anticipation', 'trust', 'surprise', 'positive', 'negative',
       'sadness', 'disgust', 'joy', 'toxic_word_count', 'left_wing',
       'righy_wing', 'Title'],
      dtype='object')

In [ ]:
df.head()

,user,post_id,comment,upvotes,responding_to,responding_to_id,comment_id,links,has_sarcasm,has_joke,...,surprise,positive,negative,sadness,disgust,joy,toxic_word_count,left_wing,righy_wing,Title
0,paleo2002,1jod9wq,"Unless they bump that up to 150 billion, this ...",9385,Aggravating_Money992,1jod9wq,mkqwfb9,[],False,False,...,NaN,0.50,0.50,NaN,NaN,NaN,0,0,0,SEC continuing $150 million lawsuit against El...
1,supercyberlurker,1jod9wq,It's.... depressing.. that I even have to ask ...,1681,Aggravating_Money992,1jod9wq,mkqw7ar,[],False,False,...,NaN,NaN,NaN,NaN,0.5,NaN,0,0,0,SEC continuing $150 million lawsuit against El...
2,Deranged_Kitsune,1jod9wq,Sounds like doge will be making another round ...,241,Aggravating_Money992,1jod9wq,mkr2i7p,[],False,False,...,0.0,0.00,0.00,0.00,0.0,0.00,0,0,0,SEC continuing $150 million lawsuit against El...
3,flyingthroughspace,1jod9wq,Didn't he just pay himself $43 billion to buy ...,682,Aggravating_Money992,1jod9wq,mkqwxan,[],False,False,...,NaN,0.25,NaN,NaN,NaN,0.25,0,0,0,SEC continuing $150 million lawsuit against El...
4,gucknbuck,1jod9wq,150 million to a billionaire is the equivalent...,198,Aggravating_Money992,1jod9wq,mkr1epy,[],False,False,...,NaN,0.25,0.25,0.25,NaN,NaN,0,0,0,SEC continuing $150 million lawsuit against El...


In [ ]:
#@title toxicity labeler model

#remove nan rows that were kepts for network / looking at nested comments
df = df[df['preprocessed_text'].notna()]

X_tfidf = tfidf.transform(df['preprocessed_text'])

# === 4. Predict ===
predictions = model.predict(X_tfidf)
probs = model.predict_proba(X_tfidf)[:, 1]

comments = df['comment'].tolist()

toxic_count = 0
for i in range(len(comments)):
    if predictions[i] == 1:
        toxic_count += 1
        print(f"\n🔴 Toxic Comment {toxic_count}:")
        print(comments[i][:300])  # Limit display to first 300 characters
        print(f"Toxicity Probability: {probs[i]:.4f}")

print(f"\n✅ Total toxic comments found: {toxic_count} out of {len(comments)}")


df['predictions'] = predictions
df['probabilities'] = probs


🔴 Toxic Comment 1:
Didn't he just pay himself $43 billion to buy his own company?
Toxicity Probability: 0.6600

🔴 Toxic Comment 2:
Gr, sock it to the Nazi asshole. He misled investors and should be held accountable.
Toxicity Probability: 0.9100

🔴 Toxic Comment 3:
Here’s comes a the let’s shut down the SEC from trump exec order
Toxicity Probability: 0.6884

🔴 Toxic Comment 4:
$150 million? Why doesn't he just pay the fine and be done with it? Spite?
Toxicity Probability: 0.6000

🔴 Toxic Comment 5:
Cost of doing business. SEC is a fucking joke.
Toxicity Probability: 0.9600

🔴 Toxic Comment 6:
"They spend their time on s--- like this when there are so many actual crimes that go unpunished," Musk said on X in January in response to the SEC's suit. Does this idiot even know what the SEC does ?
Toxicity Probability: 0.5800

🔴 Toxic Comment 7:
*GASP* No! Not one hundred fifty *million*! How will the poor man eat? Oh, the humanity!
Toxicity Probability: 0.5400

🔴 Toxic Comment 8:
Those lawye

In [ ]:
import numpy as np
import pandas as pd
import ast
from scipy.stats import mode
from tensorflow.keras.models import load_model

# Assume 'df' is already loaded and contains a column 'bert_embed'.
# Convert the embeddings to a NumPy array.
# This step handles cases where embeddings are stored as a string.
if isinstance(df.iloc[0]['bert_embed'], str):
    embeddings_array = np.stack(
        df['bert_embed'].apply(lambda x: np.array(ast.literal_eval(x), dtype=np.float32))
    )
else:
    embeddings_array = np.vstack(df['bert_embed'].values)

# Print the shape for debugging.
print("Embeddings shape before reshaping:", embeddings_array.shape)

# Ensure embeddings_array is two-dimensional.
# For example, a single embedding with shape (312,) should be reshaped to (1, 312).
if embeddings_array.ndim == 1:
    embeddings_array = embeddings_array.reshape(1, -1)

print("Embeddings shape after reshaping:", embeddings_array.shape)

# -----------------------
# Load the Pretrained Models
# -----------------------
model_files = [
    "ensemble_modelv2_1.h5",
    "ensemble_modelv2_2.h5",
    "ensemble_modelv2_3.h5",
    "ensemble_modelv2_4.h5",
    "ensemble_modelv2_5.h5",
    "ensemble_modelv2_6.h5",
    "ensemble_modelv2_7.h5",
    "ensemble_modelv2_8.h5",
    "ensemble_modelv2_9.h5"
]

# Load each Keras model.
models = [load_model(mf) for mf in model_files]

# -----------------------
# Define the Ensemble Prediction Function
# -----------------------
def ensemble_predict(models, data):
    """
    Given a list of Keras models and a NumPy array of input data, this function
    computes predictions for each model and performs majority voting on these outputs.

    Parameters:
      models (list): List of loaded Keras models.
      data (np.ndarray): Input data array of shape (n_samples, feature_dim).

    Returns:
      np.ndarray: The ensemble predictions as a 1D array.
    """
    predictions = []
    for model in models:
        # Predict on the given data.
        y_pred = model.predict(data)
        # If model predicts probabilities, convert them to class labels.
        if y_pred.ndim == 2:
            y_pred = np.argmax(y_pred, axis=1)
        predictions.append(y_pred)

    # Stack predictions to shape (n_models, n_samples) and compute the majority vote.
    preds_array = np.array(predictions)
    ensemble_preds, _ = mode(preds_array, axis=0)
    return ensemble_preds.flatten()

# -----------------------
# Generate Ensemble Predictions and Attach to DataFrame
# -----------------------
ensemble_preds = ensemble_predict(models, embeddings_array)
df["predicted_label"] = ensemble_preds

# Display the first few records with predictions.
print(df[["predicted_label"]].head())


Embeddings shape before reshaping: (15603, 312)
Embeddings shape after reshaping: (15603, 312)


488/488 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
488/488 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
488/488 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
488/488 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
488/488 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
488/488 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
488/488 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
488/488 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
488/488 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
   predicted_label
0                2
1                2
2                2
3                2
4                2


In [ ]:

# Change display settings to show all rows
pd.set_option('display.max_rows', None)



In [ ]:
mapping = {2: "Right", 1: "Left", 0: "Centre"}
df['predicted_label'] = df['predicted_label'].map(mapping)

In [ ]:
df['predicted_label'].value_counts()

,count
predicted_label,
Right,15603


In [ ]:
df.head()

,user,post_id,comment,upvotes,responding_to,responding_to_id,comment_id,links,has_sarcasm,has_joke,...,sadness,disgust,joy,toxic_word_count,left_wing,righy_wing,Title,predictions,probabilities,predicted_label
0,paleo2002,1jod9wq,"Unless they bump that up to 150 billion, this ...",9385,Aggravating_Money992,1jod9wq,mkqwfb9,[],False,False,...,NaN,NaN,NaN,0,0,0,SEC continuing $150 million lawsuit against El...,0,0.060000,Right
1,supercyberlurker,1jod9wq,It's.... depressing.. that I even have to ask ...,1681,Aggravating_Money992,1jod9wq,mkqw7ar,[],False,False,...,NaN,0.5,NaN,0,0,0,SEC continuing $150 million lawsuit against El...,0,0.080000,Right
2,Deranged_Kitsune,1jod9wq,Sounds like doge will be making another round ...,241,Aggravating_Money992,1jod9wq,mkr2i7p,[],False,False,...,0.00,0.0,0.00,0,0,0,SEC continuing $150 million lawsuit against El...,0,0.130741,Right
3,flyingthroughspace,1jod9wq,Didn't he just pay himself $43 billion to buy ...,682,Aggravating_Money992,1jod9wq,mkqwxan,[],False,False,...,NaN,NaN,0.25,0,0,0,SEC continuing $150 million lawsuit against El...,1,0.660000,Right
4,gucknbuck,1jod9wq,150 million to a billionaire is the equivalent...,198,Aggravating_Money992,1jod9wq,mkr1epy,[],False,False,...,0.25,NaN,NaN,0,0,0,SEC continuing $150 million lawsuit against El...,0,0.010000,Right
